In [ ]:
# moduli usati
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import numpy  as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
data_folder = Path("/content/drive/MyDrive/data")

# Leggiamo i dati meteo

In [ ]:
mt = pd.read_csv(data_folder / "mt/processed/t0408_mezzolombardo.csv", parse_dates=["Data"])
mt = mt.dropna()

mt

## Qualità del dato

Andiamo a vedere qual è la qualità dei dati nel nostro dataset secondo la seguente legenda.

* 1 - Dato buono
* 140 - dato incerto, per analisi climatiche o malfunzionamento stazione
* 145 - da teletrasmissione, non validato

In [ ]:
mt.Qualità.value_counts()

In [ ]:
mt.groupby([mt.Data.dt.year, "Variabile", mt.Qualità]).count()

## Grafici

Teniamo solo i dati di qualità pari a 1.

In [ ]:
mt = mt[mt.Qualità == 1]

Grafichiamo le variabili  temperatura e precipitazione. Si riporta parte del codice di seguito:

```
mt[(mt.Variabile == "Precipitazione") & (mt.Data.dt.year==2021)].plot(x="Data", y="Valore", title="Precipitazione", ylabel="mm")
```

Possiamo anche osservare l'andamento dei valori medi (mean) o cumulati (sum) giornalieri (D), settimanali (W), mensili (M) o annuali (A).
```
mt[(mt.Variabile == "Precipitazione") & (mt.Data.dt.year==2021)].set_index("Data").resample("D").sum().reset_index().plot(x="Data", y="Valore", title="Precipitazione", ylabel="mm")
```

# Previsione del valore medio giornaliero di PM10 unendo il dataset meteo e il dataset APPA

## Costruzione del dato

### Ottengo il dato giornaliero

I dati meteo possono essere utilizzati all'interno della nostra matrice X del modello di machine learning come nuove features.

Calcoliamo i dati giornalieri scegliendo i mm totali per la pioggia e la temperatura media salvandoli nella tabella "weather".

In [ ]:
# define how to aggregate data
agg = {"Precipitazione": "sum", "Temperatura": "mean"}

In [ ]:
# generate daily dataset
weather = (
    mt[["Data", "Valore", "Variabile"]]
    .dropna()
    .pivot_table(index="Data", columns="Variabile", values="Valore")
    .resample("D")
    .agg(agg)
    .sort_index()
)
weather

### Leggiamo i dati APPA

Riprendo la lettura dei dati APPA dal notebook della scorsa lezione. Carico i dati di APPA, seleziono la stazione e calcolo i valori medi orari per ogni inquinante in modo da ottenere la tabella usata per il calcolo della correlazione nella scorsa lezione.

In [ ]:
appa = pd.read_csv(
    data_folder / "DataSet_APPA/processed/appa_data.csv", parse_dates=["Data"]
)
# definiamo la stazione da analizzare
station = "Piana Rotaliana"
exp_data = appa.loc[(appa.Stazione == station)].set_index("Data").groupby(["Inquinante"]).resample("D").mean("Valore").reset_index()
# converto i nomi degli inquinanti in codici (formule chimiche o abbreviazioni)
exp_data["Inquinante"] = exp_data.Inquinante.map(
    {"PM10": "PM10", "Biossido di Azoto": "NO2", "Ozono": "O3"}
)
# converto il dataframe in una tabella pivot (formato wide) e ordino i dati per data
exp_data = exp_data.pivot_table(
    index="Data", columns="Inquinante", values="Valore"
).sort_index()

pollutant = "PM10"

exp_data

### Unione dei dati meteo e sull'inquinamento

Costruisco un'unica tabella concatenando le features che ritengo utili partendo dal modello della lezione scorsa e dalle osservazione sui dati meteo.

Creo le serie per le variabili che ritengo utili al modello:
* serie della temperatura `temp_series = weather["Temperatura"]`
* serie delle precipitazioni `rain_series = weather["Precipitazione"]`
* serie degl'inquinanti `pollutant_series = exp_data["PM10"]`

In [ ]:
temp_series = weather["Temperatura"].sort_index()
rain_series = weather["Precipitazione"].sort_index()
pollutant_series = exp_data["PM10"].sort_index()
temp_series

Creo la tabella con i vari ritardi, medie mobili, minimi, massimi e informazioni sulla data.

Si ricordano i seguenti metodi:
* una feature (colonna) di dati sfasati di un certo intervallo temporale, ad esempio 1 giorno,
`pollutant_series.shift(1).rename("inquinante_ritardato_1d"),`
* una feature (colonna) di valori medi `.mean()`, massimi `.max()` o minimi `.min()` della settimana precedente (7 giorni).
`pollutant_series.shift(1).rolling(7).mean().rename("inquinante_medio_settimana"),`
* informazioni sulla data come ad esempio giorno (day), settimana (week), mese (month), anno (year)
`pd.Series(pollutant_series.index.year, index=pollutant_series.index).rename("year")`

**Osserviamo** che la temperatura disponibile per la predizione è quella del giorno precedente visto che nel momento dell'analisi **il dato non è ancora disponibile**.

Proviamo a metterci nelle condizioni più vicine alla realtà nel momento della previsione.

Esempio di dataset con possibili features:



```
lagged_df = pd.concat(
    [   pollutant_series, # obiettivo
        temp_series.shift(2).rename("temperatura_ritardata_2d"),
        temp_series.shift(1).rename("temperatura_ritardata_1d"),
        pollutant_series.shift(1).rename("inquinante_ritardato_1d"),
        pollutant_series.shift(2).rolling(7).mean().rename("inquinante_medio_settimana"),
        pd.Series(pollutant_series.index.year, index=pollutant_series.index).rename("year"),
        pd.Series(pollutant_series.index.month, index=pollutant_series.index).rename("month"),
        pd.Series(pollutant_series.index.day, index=pollutant_series.index).rename("day"),
    ],
    axis="columns",
)
```



Usando uno dei comandi seguenti, controlla che la tabella dei dati ottenuta sia coerente con i dati iniziali.

`lagged_df.head()`

`lagged_df.tail()`



### Separazione colonna target e suddivisione dei dati (data splitting)

Prendendo spunto dalla scorsa lezione, separa la colonna PM10 dalle altre colonne che costituiscono la tabella di input del modello.

Sempre ispirandoti alla lezione precedente, suddividi i dati in due insiemi: train e test.

### Training del modello

Allena il modello.

### Previsione sul test

Elabora la previsione sul 20% di test.

### Analisi dei risultati ottenuti

Analizza i risultati sia graficamente che numericamente.

# Osservazioni

Riporta le tue osservazioni.